In [294]:
import numpy as np
from scipy.optimize import minimize
np.set_printoptions(threshold='nan')

## prepare data set
- this bloc is for preparing the data matrix has_rating[i,j] 1 if a paticular user has rated a particular movie, 0. otherwise rating[i,j] real rating that given by particular users **value exist if and only if coresponding has_rating[i,j]==1
- number of users can be optain by len(names) and number of movies can be optain by len(movies)

In [295]:
import StringIO
import csv
with open('resource - Sheet1 (2).csv') as f:
    s = StringIO.StringIO(f.read()) 
    reader = csv.reader(s, delimiter=',')
    raw = [row for row in reader]

def get_names(raw):
    return raw[2][2:]

def get_movies(raw):
    movies = []
    for i in range(3, len(raw)):
        movies.append(raw[i][1])
    return movies

def get_rating(raw):
    
    names = get_names(raw)
    movies = get_movies(raw)
    name_offset = 2
    movies_offset = 3
    
    n_names = len(names)
    n_movies = len(movies)

    has_rating = np.zeros((n_names, n_movies))
    rating = np.zeros((n_names, n_movies))
    
    for ir in range(n_movies):
        
        this_row = raw[movies_offset + ir]

        for ip in range(n_names):
            
            this_cell =  this_row[name_offset + ip]
            hr = this_cell.isdigit()
            has_rating[ip, ir] = 1 if hr else 0
            if hr:
                rating[ip, ir] = int(this_cell)
                
    return names, movies, has_rating, rating

names, movies, has_rating, rating = get_rating(raw)

print "name", names
print "movies", movies
print "has_rating", has_rating.T
print "rating", rating.T

name ['Alice', 'Bob', 'Carol', 'Dave', 'eve']
movies ['love', 'Romance', 'Cute', 'Nonstop', 'sword']
has_rating [[ 1.  1.  1.  1.  0.]
 [ 1.  0.  0.  1.  0.]
 [ 0.  1.  1.  0.  0.]
 [ 1.  1.  1.  1.  0.]
 [ 1.  1.  1.  0.  0.]]
rating [[ 5.  5.  0.  0.  0.]
 [ 5.  0.  0.  0.  0.]
 [ 0.  4.  0.  0.  0.]
 [ 0.  0.  5.  4.  0.]
 [ 0.  0.  5.  0.  0.]]


## Implementation of Detail Mean Normalization
- the seection below here is implemented to handle the case of newly added user that has never rated any movie before 
- notice that in our data set we have Eve who never rate any movie so we can think of her as newly added user

In [296]:
# init the average matrix
average = np.zeros((len(movies),1))

# find average value for each row 
for i in range (0,len(movies)):
    sum_each_row = 0
    count_num = 0
    
    for j in range (0,len(names)):
        
        if has_rating.T[i][j] == 1:
            sum_each_row = sum_each_row + rating.T[i][j]
            count_num = count_num+1
            
    average[i][0] = sum_each_row/count_num
    
print average

[[ 2.5       ]
 [ 2.5       ]
 [ 2.        ]
 [ 2.25      ]
 [ 1.66666667]]


# running data after Implementation of Detail Mean Normalization
- on the section below here we re-run our main again with the new rating matrix  

In [297]:
print "original rating",rating.T
# re-init value for rating matrix
for i in range (0,len(movies)):
    for j in range (0,len(names)):
        if has_rating.T[i][j] == 1:
            rating.T[i][j] = rating.T[i][j] - average[i]
print "rating after DMN",rating.T 

original rating [[ 5.  5.  0.  0.  0.]
 [ 5.  0.  0.  0.  0.]
 [ 0.  4.  0.  0.  0.]
 [ 0.  0.  5.  4.  0.]
 [ 0.  0.  5.  0.  0.]]
rating after DMN [[ 2.5         2.5        -2.5        -2.5         0.        ]
 [ 2.5         0.          0.         -2.5         0.        ]
 [ 0.          2.         -2.          0.          0.        ]
 [-2.25       -2.25        2.75        1.75        0.        ]
 [-1.66666667 -1.66666667  3.33333333  0.          0.        ]]


## Getting the best weight
- start Linear Regression class below here

In [298]:
class LinearRegression: 
    
     # init all the data so it can easily accessed by all methods
    def __init__(self,has_rating,real_rating,fea,lamda,names,movies,theta,x):
        
        #has_rating and real_rating here are already transposed
        self.has_rating = has_rating
        self.real_rating = real_rating
        
        self.fea=fea
        self.lamda=lamda
        self.names = names
        self.movies = movies
        self.num_names = len(names)
        self.num_movies = len(movies) 
        self.theta = theta
        self.x = x
    
    def sum_x(self,i,k,lamda):
        
        sum_x_value = 0 
        
        for j in range (0,self.num_names):
            if self.has_rating[i][j] == 1:
                sum_x_value = sum_x_value+(np.dot(self.theta[j].T,self.x[i]) - self.real_rating[i][j])*self.theta[j][k]
     
        #print "x", sum_x_value      
        return sum_x_value
    
    
    def sum_theta(self,j,k,lamda):
        
        sum_theta_value = 0
        
        for i in range (0,self.num_movies):
            if self.has_rating[i][j] == 1:
                sum_theta_value = sum_theta_value+(np.dot(self.theta[j].T,self.x[i]) - self.real_rating[i][j])*self.x[i][k]
        #print "theta", sum_theta_value
        return sum_theta_value
                
                
                
                
    #this function will seek for the x and thata that can meet the local minimum of the function
    def gradient_descent(self,alpha,lamda):
        
        # init keep_array for both x and theta
        keep_x = np.zeros((self.num_movies,self.fea))
        keep_theta = np.zeros((self.num_names,self.fea))
        keep_x = self.x
        keep_theta = self.theta
        print keep_theta
        print keep_x
        print self.real_rating
        for times in range(0,100000):
            #print "time", times
            #re-assign value to self.x and self.theta
            self.x = keep_x
            self.theta = keep_theta
            
            #set the value of x
            for i in range (0,self.num_movies):
                for k in range (0,self.fea):
                    
                    keep_x[i][k] = self.x[i][k] - alpha*(self.sum_x(i,k,lamda)+lamda*self.x[i][k])
        
            #return self.x
            
            
            #set the value of theta
            for j in range (0,self.num_names):
                for k in range (0,self.fea):
                    
                    keep_theta[j][k] = self.theta[j][k] - alpha*(self.sum_theta(j,k,lamda)+lamda*self.theta[j][k])
            
        return self.x,self.theta

In [299]:
number_feature = 2

guess_theta = np.random.uniform(0.01,1,[len(names),number_feature])
guess_x = np.random.uniform(0.01,1,[len(movies),number_feature])

linear_obj = LinearRegression(has_rating.T,rating.T,number_feature,0.01,names,movies,guess_theta,guess_x)
result_x, result_theta = linear_obj.gradient_descent(0.05,0.01)

print "x", result_x
print "theta", result_theta

[[ 0.85886866  0.5741266 ]
 [ 0.12658771  0.20047575]
 [ 0.77902511  0.73879396]
 [ 0.36389442  0.57628269]
 [ 0.8632574   0.95311176]]
[[ 0.78548495  0.21552536]
 [ 0.16604414  0.70160449]
 [ 0.13500897  0.55218919]
 [ 0.90233916  0.56086238]
 [ 0.34622078  0.35528929]]
[[ 2.5         2.5        -2.5        -2.5         0.        ]
 [ 2.5         0.          0.         -2.5         0.        ]
 [ 0.          2.         -2.          0.          0.        ]
 [-2.25       -2.25        2.75        1.75        0.        ]
 [-1.66666667 -1.66666667  3.33333333  0.          0.        ]]
x [[-0.00608793 -1.62386806]
 [ 0.02689458 -1.61890727]
 [-0.03044228 -1.28336974]
 [ 0.49122889  1.36316579]
 [ 1.49373712  0.82644061]]
theta [[ -2.63560219e-01  -1.54417118e+00]
 [ -2.63607941e-01  -1.54436941e+00]
 [  1.37858915e+00   1.52701104e+00]
 [ -6.59380247e-01   1.53126319e+00]
 [  1.64431766e-22   1.81547069e-22]]


In [300]:
out = np.zeros((len(movies),len(names)))
for i in range(0,len(movies)):
            for j in range(0,len(names)):
                    out[i][j] = float("{0:.2f}".format(float((np.dot(result_theta[j].T,result_x[i])))+float((average[i]))))
print out

[[ 5.01  5.01  0.01  0.02  2.5 ]
 [ 4.99  4.99  0.06  0.    2.5 ]
 [ 3.99  3.99 -0.    0.05  2.  ]
 [ 0.02  0.02  5.01  4.01  2.25]
 [-0.   -0.    4.99  1.95  1.67]]
